# **Atenção**

## Para executar esse notebook utilize o T4 GPU

Além disso, a saída de algumas células estão comentadas, visto que sua exibição implica na utilização de memória RAM do Colab, gerando erro de processamento. Os códigos estão funcionando, porém precisam de maior capacidade computacional para rodarem sem problemas. Na próxima sprint espera-se conseguir aprimorar as questões de eficiência.

# Integrantes

- Dayllan Alho
- Giovanna Furlan
- Luiz Ferreira
- Maria Luísa Maia
- Thainá Lima
- Vinicius Fernandes

# Introdução

## **Problema a ser resolvido**

O projeto tem como objetivo desenvolver uma aplicação para a Aegea, uma empresa de saneamento, que melhore a detecção de fraudes no consumo de água. Essas fraudes, como a manipulação de hidrômetros e ligações clandestinas, não só prejudicam o faturamento e a qualidade do serviço, mas também podem causar danos à infraestrutura e representar riscos à saúde pública.

## **Solução Proposta**

Para identificar as fraudes, foi proposto o desenvolvimento de uma aplicação baseada em Machine Learning que melhore a capacidade da Aegea de detectar comportamentos fraudulentos no consumo de água. Essa solução considera a influência de variáveis externas, como fatores econômicos, climáticos e geográficos, para aumentar a assertividade na identificação e melhorar os processos de negócio da empresa.

# Setup

A configuração de setup é o processo de preparar e organizar o ambiente para uso. Envolvendo a instalação de bibliotecas e configuração de outros ajustes necessários. O objetivo é criar um ambiente funcional para executar tarefas específicas.

## Instalação das bibliotecas

 Nesta seção, são instaladas e importadas as bibliotecas necessárias para a manipulação, análise e visualização dos dados.

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import gdown

## **Importação da base de dados**

Para realizar a análise, padronização e manipulação dos dados é necessário selecionar a base de dados desejada. Neste documento a importação da mesma será feita através do gdown e o arquivo está em formato CSV.

In [ ]:
arquivo_base = "{}.csv"
ids = {
    "CONSUMO_GERAL": "1-IOqfwmh_tTIDHeOer8J-HkGFtwuX67g",
    "FRAUDES": "1-MbIlChqQapcxFkoJgpbQIsN9FBLfbX1",
}
for key, file_id in ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    arquivo_destino = arquivo_base.format(key)
    gdown.download(url, arquivo_destino, quiet=False)

In [ ]:
FRAUDES = pd.read_csv("./FRAUDES.csv", delimiter=",")
CONSUMO_GERAL = pd.read_csv("./CONSUMO_GERAL.csv", delimiter=",")

## One hot Encoding

Uma técnica usada em machine learning para transformar variáveis categóricas em uma forma que os algoritmos possam entender. Cada categoria é convertida em uma coluna binária (0 ou 1), representando a presença ou ausência dessa categoria em uma amostra. É importante porque muitos algoritmos não conseguem trabalhar diretamente com dados categóricos. Ao transformar as categorias em números binários, o modelo pode processar as informações corretamente, melhorando a precisão das previsões.

In [ ]:
class OHC_trasformation:
    ohc_encoders = {}

    @classmethod
    def encode(cls, df, columns_name):
        for column_name in columns_name:
            # Instanciando o OneHotEncoder
            ohc = OneHotEncoder(handle_unknown="ignore", sparse_output=False).set_output(transform="pandas")
            # Fit e transformação dos dados
            ohc_fit_transform = ohc.fit_transform(df[[column_name]])
            # Salvando o encoder para futuras utilizações
            cls.ohc_encoders[column_name] = ohc
            # Dropando a coluna original
            df.drop(columns=[column_name], inplace=True)
            # Concatenando o DataFrame original com as colunas One-Hot Encoded
            df = pd.concat([df, ohc_fit_transform], axis=1)

        return df

# Aplicar o One Hot Encoding no novo ambiente
CONSUMO_GERAL = OHC_trasformation.encode(CONSUMO_GERAL, ["CATEGORIA", "TIPO_LIGACAO"])
FRAUDES = OHC_trasformation.encode(FRAUDES, ['STATUS', 'ATRASO'])


A classe `TableTransformation` é responsável por transformar um DataFrame usando operações como pivotamento de dados e seleção de colunas estáticas. Isso é útil para reorganizar os dados em formato de tabela e trabalhar com colunas específicas.

1. **pivot_data**: Realiza o pivotamento dos dados, utilizando uma chave primária como índice e uma coluna de datas como cabeçalhos. Valores nulos são substituídos por zero.
2. **trasform_estatic_data**: Seleciona colunas estáticas, exclui a coluna de valores pivotados, remove duplicatas e ordena pela chave primária.


In [ ]:
class TableTransformation:
    def __init__(self, nome_coluna_chave_primaria, nome_coluna_data, nome_coluna_contem_value):
        self.nome_coluna_data = nome_coluna_data
        self.nome_coluna_chave_primaria = nome_coluna_chave_primaria
        self.nome_coluna_contem_value = nome_coluna_contem_value

    def pivot_data(self, df):
        # Realizando o pivotamento dos dados em Pandas
        df_pivot = df.pivot(index=self.nome_coluna_chave_primaria, 
                            columns=self.nome_coluna_data, 
                            values=self.nome_coluna_contem_value).fillna(0)
        return df_pivot

    def trasform_estatic_data(self, df):
        # Selecionando as colunas estáticas (incluindo a coluna chave primária)
        colunas_estaticas = df.drop(columns=[self.nome_coluna_contem_value]).drop_duplicates()

        # Ordenando pelo identificador chave primária (MATRICULA)
        colunas_estaticas = colunas_estaticas.sort_values(self.nome_coluna_chave_primaria)

        return colunas_estaticas

<details>
<summary style="font-size:1.5em;">Instanciação da Classe e Execução das Transformações</summary>
<br>

Neste trecho de código, a classe `TableTransformation` é instanciada e utilizada para realizar transformações em um DataFrame. O processo envolve pivotamento de dados, extração de colunas estáticas, verificação de chaves primárias, e combinação final dos dados.

In [ ]:
# Instanciação da classe e execução das transformações
tt = TableTransformation("MATRICULA", "REFERENCIA", "CONS_MEDIDO")

# Realizando o pivotamento no DataFrame CONSUMO_GERAL
df_pivot = tt.pivot_data(CONSUMO_GERAL)

# Extraindo as colunas estáticas
colunas_estaticas = tt.trasform_estatic_data(CONSUMO_GERAL)

# Verificando se a chave primária "MATRICULA" está presente e se o índice é único
if "MATRICULA" in colunas_estaticas.columns:
    colunas_estaticas.set_index("MATRICULA", inplace=True)

# Verificando se os índices de ambos os DataFrames são únicos
if not df_pivot.index.is_unique or not colunas_estaticas.index.is_unique:
    print("Os índices não são únicos. Resetando o índice para evitar conflitos.")
    df_pivot.reset_index(inplace=True)
    colunas_estaticas.reset_index(inplace=True)

# Concatenando os dados pivotados com as colunas estáticas
df_final = pd.concat([df_pivot, colunas_estaticas], axis=1)

# Exibindo o resultado final
print("\nDataFrame Final (Dados Pivotados + Colunas Estáticas):")
display(df_final.head())

# Salvando o DataFrame final em um arquivo CSV
df_final.to_csv('resultado_transformacao.csv', index=False)

# Mensagem de confirmação
print("O DataFrame final foi salvo como 'resultado_transformacao.csv'.")

<details>
<summary style="font-size:1.5em;">Transformações sem Normalização</summary>

<br>Neste estágio do pipeline de transformação, optamos por não aplicar normalização nos dados. O objetivo é observar como o DataFrame, composto por dados pivotados e colunas estáticas, se comporta diretamente em modelos subsequentes. Ao não normalizar, é possível testar a sensibilidade do modelo a diferentes magnitudes de variáveis, identificando padrões ou problemas específicos de escalas.

Este DataFrame foi salvo como `resultado_transformacao.csv` para ser utilizado nas etapas seguintes do projeto.
</details>
